# Model parameters and mutation effects (EVmutation)

## Content

This notebook demonstrates how to explore the parameters of undirected graphical models inferred using EVcouplings, and how to use those to quantitatively predict the effects of mutations.

## Reference

Hopf, T. A., Ingraham, J. B., Poelwijk, F.J., Schärfe, C.P.I., Springer, M., Sander, C., & Marks, D. S. (2017). Mutation effects predicted from sequence co-variation. *Nature Biotechnology* **35**, 128–135 doi:10.1038/nbt.3769

## Tutorial

### Part 1: Load the model parameters from file

This file is generated by the couplings stage of the pipeline (using plmc) and has the extension .model

In [54]:
from evcouplings.couplings import CouplingsModel

In [55]:
# load parameters from file to create a pairwise model
c = CouplingsModel("example/PABP_YEAST.model_params")

### Part 2: Predict mutation effects

In [56]:
import pandas as pd
from evcouplings.mutate import predict_mutation_table, single_mutant_matrix

In [57]:
# read the experimental mutational scanning dataset for PABP by Melamed et al., RNA, 2013
data = pd.read_csv(
    "example/PABP_YEAST_Fields2013-singles.csv", sep=";", comment="#"
)

# predict mutations using our model
data_pred = predict_mutation_table(
    c, data, "effect_prediction_epistatic"
)

In [58]:
# can also add predictions by the corresponding independent model
c0 = c.to_independent_model()

data_pred = predict_mutation_table(
    c0, data_pred, "effect_prediction_independent"
)

In [59]:
data_pred.head()

,mutant,linear,log,effect_prediction_epistatic,effect_prediction_independent
0,G126A,0.711743,-0.490571,-2.610615,0.406487
1,G126C,0.449027,-1.155127,-5.663638,-0.027602
2,G126E,0.588928,-0.763836,-6.611062,-1.827570
3,G126D,0.229853,-2.121218,-7.270577,-1.180076
4,G126N,0.679435,-0.557593,-5.809167,0.387440


#### Predicting single-substitution landscape (independent of experiment) 

In [60]:
singles = single_mutant_matrix(
    c, output_column="effect_prediction_epistatic"
)

singles.head()

,mutant,pos,wt,subs,frequency,column_conservation,effect_prediction_epistatic
0,K123A,123,K,A,0.077201,0.112437,0.796669
1,K123C,123,K,C,0.001461,0.112437,-3.337328
2,K123D,123,K,D,0.118235,0.112437,-0.316713
3,K123E,123,K,E,0.110503,0.112437,-1.078200
4,K123F,123,K,F,0.007791,0.112437,-3.013274


#### Predicting arbitary mutations

In [61]:
# double mutant L186M, G188A
delta_E, delta_E_couplings, delta_E_fields = c.delta_hamiltonian([(186, "L", "M"), (188, "G", "A")])
delta_E

-6.9225586684769951

#### Short-cuts to single and double mutations

In [62]:
# single mutation matrix
c.smm(127, "E")

-7.6052584765675419

In [63]:
# double mutation matrix (double mutant L186M, G188A)
c.dmm(186, 188, "M", "A")

-6.9225586684769951

#### Statistical energy of a sequence (rather than delta to WT)

In [64]:
E, E_couplings, E_fields = c.hamiltonians([c.seq()])[0]
E

312.19741128035912

### Part 3: Explore model parameters

Please see the documentation of evcouplings.couplings.model.CouplingsModel for all available methods. Note that most of these methods can be accessed using lists of positions/symbols. 

Examples include:

In [65]:
# full sequence
print(c.seq())

# symbol for particular position (or list of positions)
print(c.seq(127))

['K' 'G' 'S' 'G' 'N' 'I' 'F' 'I' 'K' 'N' 'L' 'H' 'P' 'D' 'I' 'D' 'N' 'K'
 'A' 'L' 'Y' 'D' 'T' 'F' 'S' 'V' 'F' 'G' 'D' 'I' 'L' 'S' 'S' 'K' 'I' 'A'
 'T' 'D' 'E' 'N' 'G' 'K' 'S' 'K' 'G' 'F' 'G' 'F' 'V' 'H' 'F' 'E' 'E' 'E'
 'G' 'A' 'A' 'K' 'E' 'A' 'I' 'D' 'A' 'L' 'N' 'G' 'M' 'L' 'L' 'N' 'G' 'Q'
 'E' 'I' 'Y' 'V' 'A' 'P' 'H' 'L' 'S' 'R']
N


In [66]:
# positions in model
print(c.index_list)

[123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140
 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158
 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176
 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194
 195 196 197 198 199 200 201 202 203 204]


In [67]:
# alphabet
print(c.alphabet)

['A' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'K' 'L' 'M' 'N' 'P' 'Q' 'R' 'S' 'T' 'V'
 'W' 'Y']


In [68]:
# pair coupling parameters
c.Jij(127, 172, c.seq(127), c.seq(172))

-0.2060956209897995

In [69]:
# field parameters
c.hi(127, c.seq(127))

0.30619758

### Part 4: Index mapping for complexes

When inferring parameters for a concatenated sequence alignment (e.g. protein complexes or other discontinuous sequence segments), the internal model numbering does not match the numbering of the actual sequence. In this case, the model numbering can be manually remapped such that the model can be indexed using tuples (segment_id, position_in_segment).

*(Note this example does not execute here and is intended as a reference of how to use the code only)*

In [70]:
from evcouplings.couplings import Segment, SegmentIndexMapper

In [ ]:
# Define which segments the concatenated sequence in the model is made of.
# Most important parameters are region_start and region_end of Segment
s_pare = Segment("aa", "F7YBW7", 1, 103, segment_id="parE")
s_pard = Segment("aa", "F7YBW8", 1, 93, segment_id="parD")

# Create index mapper
mapper = SegmentIndexMapper(True, 1, s_pare, s_pard)

# Update indices in model to complex numbering
c_mapped = mapper.patch_model(c, inplace=False)

# Now access model using tuples of indices rather than single positions alone
print(c.seq(("parD", 59)))
print(c.smm(("parD", 59), "A"))